### Сетап

In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from datasets import load_dataset

In [2]:
DATASET_NAME = "kuznetsoffandrey/sberquad"
MODEL_SAVE_DIR = './distilbert-v2'

model = AutoModelForQuestionAnswering.from_pretrained(MODEL_SAVE_DIR)
tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_DIR)
dataset_test = load_dataset(DATASET_NAME, split="test")

### Проверка работы модели на части тестовых данных

In [3]:
import random

dataset_test = list(dataset_test)
random.shuffle(dataset_test)

In [4]:
for example in dataset_test[:5]:
    question = example['question']
    context = example['context']
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)

    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)

    answer_tokens = inputs['input_ids'][0][start_idx: end_idx + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    print(f"Контекст: {context}")
    print(f"Вопрос: {question}")
    print(f"Ответ: {answer}\n")

Контекст: В античности территория Каринтии составляла древнеримскую провинцию Норик. В VI веке началось заселение области славянами, которых вскоре подчинил себе Аварский каганат. Позднее эти земли входили в состав раннефеодальных славянских государств Само и Карантания. После короткого периода баварской гегемонии Каринтия влилась в государство Каролингов, а при его распаде была расчленена на несколько марок во главе с немецкими графами. В 870-х гг. их объединил под своим скипетром будущий император Арнульф Каринтийский.
Вопрос: В какое государство влилась Каринтия после короткого периода баварской гегемонии?
Ответ: Каролингов

Контекст: В философии под эффективностью понимается способность действующей причины произвести определённый эффект. Поскольку относительно того, что является способностью, не существует единого мнения, поэтому возможны различные спорные случаи, касающиеся того, что именно существует и что именно проявляет свою истинную причинность. Проблема в том, что часто пред